# Demostración Curvas Elípticas

Primero, escogemos la curva Curve25519 recomendada en el RFC7748 (https://tools.ietf.org/html/rfc7748#page-4).
Damos el cuerpo, la curva y el punto G base del subgrupo (también recomendado en el artículo).

In [1]:
K = GF(2^(255)-19); # Cuerpo
E =  EllipticCurve(K, [0, 486662, 0, 1, 0]); # Curva
G = E(9,14781619447589544791020593568409986887264606134616475288964881837755586237401); # Generador del subgrupo
n = 2^252 + 0x14def9dea2f79cd65812631a5cf5d3ed; # Orden subgrupo generado por G

Esta el la función para generación de la llave privada (número) y la pública (punto)

In [2]:
def obtenerClaves(G, n):
    d = ZZ.random_element(1, n-1)
    H = d*G
    
    return d,H #Privada / Pública 

Obtenemos una pareja de claves

In [3]:
dA, HA = obtenerClaves(G, n); dA, HA

(955420798291222162383239997443926117836422561522270142103612702475525932480,
 (5121923852907061438581061465499853009050777329813211467055215122507590041839 : 3409004576848160780621935864572296721639491914975125550113634936524990286987 : 1))

## ECDSA

Función para firmar un mensaje y otra para verificar la firma

In [4]:
def firmarEC(mensaje, G, n, d): #Menasaje / Generador subgrupo / Orden subgrupo / Clave privada
    z = hash(mensaje) % n; # Hash del mensaje
    
    k = ZZ.random_element(1, n-1)
    P = k*G; 
    r = P[0]; 
    r = Integer(r) % n;
    k_inv = inverse_mod(k,n);
    s = k_inv*(z+r*d) % n;

    return r, s

def verificarFirmaEC(mensaje, G, n, H, r, s): #Mensaje recibido / Generador subgrupo / Orden subgrupo / clave pública / valores firma (r, s)
    z = hash(mensaje) % n; # Hash del mensaje
    
    u1 = ((1/s)*z) % n; 
    u2 = ((1/s)*r) % n; 
    P1 = u1*G + u2*H; 
    xP = Integer(P1[0]) % n; 
    
    igual = False
    if r == xP:
        igual = True
    
    return igual

Pongamos ahora en práctica el métdodo implementado. Supongamos que Alice desea enviar un mensaje a Bob. Decide firmarlo y enviárselo. (La parte de encriptar el archivo no lo tenemos en cuenta, solo nos interesa la firma)

In [18]:
mensaje = "Bob, me comprado un gatito. Puedes venir a verlo esta tarde"

In [19]:
r, s = firmarEC(mensaje, G, n, dA)

Suponemos que Bob tras descifrar el mensaje obtiene el siguiente y decide ver si verdaderamente lo ha enviado Alice.

In [20]:
mensajeDescifrado = "Bob, me comprado un gatito. Puedes venir a verlo esta tarde"
verificarFirmaEC(mensajeDescifrado, G, n, HA, r, s)

True

Bob comprueba que efectivamente se lo ha enviado Alice y se organizar para ver al gatito. Alice, se da cuenta de que no han especificado hora y le envía otro mensaje para concretar. 

In [23]:
mensaje2 = "Nos vemos en mi casa a las 14:00"
r2, s2 = r, s = firmarEC(mensaje2, G, n, dA)

Carol, está eenvidiosa con ambos porque ella también quiere ver al gatito. Intercepta el mensaje enviado por Alice y lo cambia por otro y se lo envía a Bob haciéndose pasar por Alice. Bob cuando descifra el mensaje obtiene

In [24]:
mensajeDescifrado2 = "Me ha surgido un problema, no puedes venir al final"
verificarFirmaEC(mensajeDescifrado2, G, n, HA, r2, s2)

False

Bob ve que el mensaje no lo ha enviado Alice por lo que no le da credibilidad al mismo